In [8]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, ShuffleSplit
from sklearn.preprocessing import *
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample
from sklearn.datasets import make_regression
from sklearn.metrics import *
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from xgboost import XGBClassifier, XGBRegressor
from imblearn.datasets import make_imbalance
import statsmodels.api as sm

In [9]:
def target_encoding(data, column, target):
    
    grouped = data[[column,target]].groupby(column,as_index=False).mean()
    empty_dict = {}
    for i in range(len(grouped)):
        empty_dict[grouped.iloc[i,0]]=grouped.iloc[i,1]
    data[column]=data[column].map(lambda x: empty_dict[x])
    
    return data

In [15]:
df=pd.read_csv('sample.csv')

In [16]:
for col in df.select_dtypes(include='O').columns:
    df[col]=target_encoding(df,col,'price')

In [17]:
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,price_bin,volume
0,0,0.23,0,0,0,61.5,55.0,326.0,0.0,38.202030
1,1,0.21,1,1,1,59.8,61.0,326.0,0.0,34.505856
2,2,0.23,2,2,2,56.9,65.0,327.0,0.0,38.076885
3,3,0.29,3,3,3,62.4,58.0,334.0,0.0,46.724580
4,4,0.31,4,4,4,63.3,58.0,335.0,0.0,51.917250


In [18]:
def upsample_classes(data, target):
    
    lst = list(data[target].unique())
    
    classes = []
    for c in lst:
        classes.append(data[data[target]==c])
    
    length = 0
    class_lab = None
    for c in classes:
        if len(c)>length:
            length=len(c)
            class_lab = c
    class_lab = class_lab[target].unique()[0]
    
    regroup = pd.concat(classes)
    maj_class = regroup[regroup[target]==class_lab]

    lst.remove(class_lab)
    
    new_classes=[]
    for i in lst:
        new_classes.append(resample(data[data[target]==i],replace=True, n_samples=len(maj_class)))

    minority_classes = pd.concat(new_classes)
    upsample = pd.concat([regroup[regroup[target]==class_lab],minority_classes])

    return upsample

In [19]:
def classification_model(X,y,method,target,test_balance = False):
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=14)
    
    train = pd.concat([X_train,y_train],axis=1)
    
    train_balanced = upsample_classes(train,target)
    
    X_train = train_balanced.drop(target, axis=1)
    y_train = train_balanced[target]
    
    if test_balance == True:
        test = pd.concat([X_test,y_test],axis=1)
        test_balanced = upsample_classes(test,  target)
        X_test = test_balanced.drop(target, axis=1)
        y_test = test_balanced[target]
    
    method.fit(X_train,y_train)
    p = method.predict(X_test)
        
    print('accuracy score')
    print(f1_score(y_test, p ,average='micro'))
    print('-'*20)
    
    plt.figure(figsize=(15,8))
    sns.heatmap(confusion_matrix(y_test,p),annot=True)
    plt.tight_layout()
    plt.show()

In [20]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [21]:
X=df.drop(['price','price_bin'],axis=1)

In [22]:
y=df.price_bin

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=14)

In [24]:
y.value_counts()

0.0    39013
1.0     9285
2.0     3574
3.0     2068
Name: price_bin, dtype: int64

In [25]:
X_train_1, y_train_1 = make_imbalance(X_train, y_train, sampling_strategy={0: 1500, 1: 1500, 2: 1500, 3: 1500},random_state=14)

In [26]:
train = pd.concat([X_train,y_train],axis=1)

train_balanced = (upsample_classes(train,'price_bin'))

X_train_2 = train_balanced.drop(['price_bin'],axis=1)
y_train_2 = train_balanced.price_bin